# 河道模型 (River Channel Model)

`河道 (RiverChannel)` 类使用线性水库模型来模拟一段河流。在这个模型中，从河段流出的水量与当前存储在其中的水量成正比。这是一种模拟河流流量演算的常用且有效的方法。本Jupyter Notebook演示了 `RiverChannel` 的设置和仿真。

## 状态变量

河道的状态由以下变量定义：

- `volume` (float): 当前存储在河段中的水量 (m³)。
- `outflow` (float): 从河段末端流出的水量 (m³/s)。

## 参数

河道的行为由一个单一参数控制：

- `k` (float): 蓄水系数 (s⁻¹)，它决定了蓄水量和出流量之间的关系 (`outflow = k * volume`)。一个较小的 `k` 值会导致更大的流量衰减和延迟。

## 仿真示例

以下示例模拟了河道对洪水波（一个高入流脉冲）的响应。仿真显示了河道在洪水波通过时如何衰减（降低洪峰）和延迟（推迟洪峰到达时间）它。为了进行比较，绘制了入流过程线和产生的出流过程线。

**注意:** 要运行此Notebook，请确保您已安装 `matplotlib` 和 `numpy` (`pip install matplotlib numpy`)，并且您是从项目的根目录运行Jupyter服务器的。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from swp.simulation_identification.physical_objects.river_channel import RiverChannel
from swp.core.interfaces import State, Parameters

# 河道的初始状态和参数
initial_state = State(volume=10000.0, outflow=0.0)
parameters = Parameters(k=0.00002) # 蓄水系数

# 创建一个RiverChannel实例
river_channel = RiverChannel(name="main_river", initial_state=initial_state, parameters=parameters)

# 仿真设置
dt = 3600  # 时间步长（秒）(1小时)
simulation_duration = 3600 * 24 * 3  # 3天
num_steps = int(simulation_duration / dt)

# 生成一个洪水波入流过程线
base_inflow = 10.0 # m³/s
peak_inflow = 100.0 # m³/s
inflow_hydrograph = base_inflow + (peak_inflow - base_inflow) * np.exp(-((np.arange(num_steps) - 24)**2) / 100))

# 存储结果用于绘图
history = []

# 运行仿真循环
for i in range(num_steps):
    inflow = inflow_hydrograph[i]
    river_channel.set_inflow(inflow)
    current_state = river_channel.step(action=None, dt=dt)
    history.append(current_state.copy())

# 打印最终状态
print("最终状态:", river_channel.get_state())

# 准备绘图数据
time_in_hours = [i * dt / 3600 for i in range(num_steps)]
volumes = [s['volume'] for s in history]
outflows = [s['outflow'] for s in history]

# 绘制结果
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.plot(time_in_hours, inflow_hydrograph, 'b--', label='入流 (Inflow)')
ax1.plot(time_in_hours, outflows, 'r-', label='出流 (Outflow)')
ax1.set_xlabel('时间 (小时)')
ax1.set_ylabel('流量 (m³/s)')
ax1.set_title('河道流量演算')
ax1.grid(True)
ax1.legend()

ax2 = ax1.twinx()
ax2.plot(time_in_hours, [v/1e6 for v in volumes], 'g:', label='蓄水量 (Storage Volume)')
ax2.set_ylabel('蓄水量 (百万 m³)', color='g')
ax2.tick_params(axis='y', labelcolor='g')
fig.tight_layout()
plt.show()
